# Lab 4: Feature Points

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

imagesDir = 'images' # Change this, according to your images' directory path

In [3]:
filename = 'corners_01.jpg'
img = cv2.imread(os.path.join(imagesDir, filename))
cv2.imshow("corners", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

og_img = img.copy() # let's copy the image to be able to use it later without having to read it again

### 1. Corner Detection

[Harris Corner Detector](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#gac1fc3598018010880e370e2f709b4345)

In [8]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

neighbourhood = 2
aperture = 3
free_param = 0.04
dst = cv2.cornerHarris(gray, neighbourhood, aperture, free_param)

# result is dilated for marking the corners, not important
dst = cv2.dilate(dst, None)

# Threshold for an optimal value, it may vary depending on the image
thr = 0.01

img[dst > thr * dst.max()] = [0, 0, 255]

cv2.imshow("Harris Corner Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[Shi-Tomasi Corner Detector](https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541)

In [9]:
max_corners = 200
quality = 0.01
mindist = 10
corners = cv2.goodFeaturesToTrack(gray, max_corners, quality, mindist, blockSize=neighbourhood, k=free_param)
corners = np.intp(corners)

img2 = og_img.copy()
for i in corners:
    x, y = i.ravel()
    cv2.circle(img2, (x, y), 3, 255, -1)

cv2.imshow("Shi-Tomasi Corner Detection", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Exercise 1.1**: Implement the [FAST Corner Detector](https://docs.opencv.org/4.x/df/d74/classcv_1_1FastFeatureDetector.html) to detect corners in images. Disable non maximum suppression and compare the results.

In [10]:
fast = cv2.FastFeatureDetector_create()

kp = fast.detect(og_img, None)
img2 = cv2.drawKeypoints(og_img, kp, None, color=(255,0,0))

fast.setNonmaxSuppression(0)
kp = fast.detect(img, None)
img3 = cv2.drawKeypoints(img, kp, None, color=(255,0,0))

cv2.imshow("First FAST Corner Detector", img2)
cv2.imshow("Second FAST Corner Detector", img3)

while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()



**Exercise 1.2**: Verify the impact of resizing the image before applying each of the corner detectors by:
 * Downsizing the image to 1/4 of its original size
 * Upsizing the image to twice its original size

In [17]:
fast = cv2.FastFeatureDetector_create()

def fast_corner_detection (img):
    kp = fast.detect(img, None)
    return cv2.drawKeypoints(img, kp, None, color=(255,0,0))

smaller_image = cv2.resize(og_img, (og_img.shape[1] // 4, og_img.shape[0] // 4))
bigger_image = cv2.resize(og_img, (og_img.shape[1] * 2, og_img.shape[0] * 2))

img1 = fast_corner_detection(smaller_image)
img2 = fast_corner_detection(bigger_image)

cv2.imshow("First FAST Corner Detector", img1)
cv2.imshow("Second FAST Corner Detector", img2)

while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()



### 2. Blob Detection

[SIFT Blob Detector](https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html)

In [4]:
new_img = cv2.imread(os.path.join(imagesDir, 'match_NotreDame_1.jpg'))
new_img = cv2.resize(new_img, (0, 0), fx=0.4, fy=0.4)
gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)

cv2.imshow("image", new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Initiate SIFT detector
sift = cv2.SIFT_create()

# Find the keypoints
kp = sift.detect(gray, None)

# Draw the keypoints (with size and orientation)
sift_img = cv2.drawKeypoints(new_img, kp, None, (-1, -1, -1), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("SIFT", sift_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

**Exercise 2.1**: Implement keypoint detection with the [Orb Blob Detector](https://docs.opencv.org/4.x/db/d95/classcv_1_1ORB.html) and compare the results with those obtained using SIFT.

In [11]:
orb = cv2.ORB_create()
keypoints_orb, descriptors_orb = orb.detectAndCompute(new_img, None)

im_with_keypoints = cv2.drawKeypoints(new_img, keypoints_orb, None, (0, 255, 0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Orb Blob", im_with_keypoints)
while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()


### 3. Matching 

Check tutorial [here](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html).

In [5]:
query = cv2.imread(os.path.join(imagesDir, 'match_box01a_1.png'), cv2.IMREAD_GRAYSCALE) # queryImage
train = cv2.imread(os.path.join(imagesDir, 'match_box01a_2.png'), cv2.IMREAD_GRAYSCALE) # trainImage

cv2.imshow("query", query)
cv2.imshow("train", train)
cv2.waitKey(0)
cv2.destroyAllWindows()

Match keypoints in images using:
 * Orb Detector for keypoint detection
 * Brute Force matcher with Hamming similarity (ideal for Orb Detector)

In [10]:
# Initiate ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(query, None)
kp2, des2 = orb.detectAndCompute(train, None)

# Create a Brute Force Matcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Match descriptors
matches = bf.match(des1, des2)

# Sort them in the order of their distance
matches = sorted(matches, key = lambda x:x.distance)

# Draw matches
match_output = cv2.drawMatches(query, kp1, train, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("matches", match_output)
while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()

**Exercise 3.1**: Replace the Orb Detector by SIFT for keypoint detection and verify the results.

In [13]:
sift = cv2.SIFT_create()

kp1, des1 = sift.detectAndCompute(query, None)
kp2, des2 = sift.detectAndCompute(train, None)

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

matches = bf.match(des1, des2)

matches = sorted(matches, key = lambda x:x.distance)

match_output = cv2.drawMatches(query, kp1, train, kp2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv2.imshow("matches", match_output)
while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()

**Exercise 3.2**: Save the top-2 best matches for each keypoint and apply Lowe's ratio test to remove outliers.

**Tips**: 
* You can find the top-k best matches for each keypoint by replacing ```match()``` with ```knnMatch()```. 
* You can draw all k matches using ```drawMatchesKnn()``` instead of ```drawMatches()```. 
* Lowe's ratio test analyses the two best matches. If the two best matches have similar distances, then both matches are removed. The best match is only saved if its distance is sufficiently smaller than the distance of the second-best match.

In [23]:
def detect_and_match(query, train, detector, ratio_test=0.75):
    kp1, des1 = detector.detectAndCompute(query, None)
    kp2, des2 = detector.detectAndCompute(train, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good.append([m])

    img_matches = cv2.drawMatchesKnn(query, kp1, train, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches


sift = cv2.SIFT_create()
img_sift = detect_and_match(query, train, sift)

orb = cv2.ORB_create()
img_orb = detect_and_match(query, train, orb)

cv2.imshow("Lowe with SIFT", img_sift)
cv2.imshow("Lowe with ORB", img_orb)
while True:
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()


**Exercise 3.3**: Replace the Brute Force algorithm by FLANN for keypoint matching.

In [7]:
def detect_and_match(query, train, detector, ratio_test=0.75):
    kp1, des1 = detector.detectAndCompute(query, None)
    kp2, des2 = detector.detectAndCompute(train, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good.append([m])

    img_matches = cv2.drawMatchesKnn(query, kp1, train, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches


sift = cv2.SIFT_create()
img_sift = detect_and_match(query, train, sift)

cv2.imshow("Lowe with SIFT", img_sift)
while True:
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()

**Exercise 3.4**: Find Wally! Given an image of Wally's profile and a puzzle where Wally is hidden, apply keypoint detection and matching algorithms to find Wally.

In [14]:
def detect_and_match(query, train, detector, ratio_test=0.75):
    kp1, des1 = detector.detectAndCompute(query, None)
    kp2, des2 = detector.detectAndCompute(train, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(des1, des2, k=2)

    good = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good.append([m])

    img_matches = cv2.drawMatchesKnn(query, kp1, train, kp2, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    return img_matches

img_wallys_face = cv2.imread("images/wally.png")
img_find_wally = cv2.imread("images/find_wally.jpg")

sift = cv2.SIFT_create()
img_sift = detect_and_match(img_wallys_face, img_find_wally, sift)

cv2.imshow("Find Wally Image", img_sift)
while True:
    if cv2.waitKey(1) == ord("q"):
        break
cv2.destroyAllWindows()